<!--NOTEBOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="./figures/LogoOpenclassrooms.png">
<font size="4">
<p>
Cette étude a été réalisée dans le cadre du 5ème projet de ma formation Datascientist dispensée en MOOC par 
**<font color='blus'>Openclassrooms / écoles Centrale-Supélec</font>**.
</p>    

<p>
Ce notebook présente un modèle de ségmentation de clients d'un site d'achats en ligne. 
</p>
<p>
Ce modèle se base sur les données fournies par le site :
</p>
<p>
https://archive.ics.uci.edu/ml/datasets/Online+Retail
</p>
<p>
</p>
</font>

# <font color='blus'> Synthèse du 12/07/2018</font>

* Modèle simplifié (sans référence aux produits) : réduction de dimension par Algo t-SNE

    * Variables qualitatives
    * Variables quantitatives
    * Aggregation des variables dans la mm structure.

* Outliers : élimination.

* Redondance : voire les corrélations.

* Méthodes pour le clustering. Comparer avec des données.
    * Kmeans 
    * GMM
    * DBSCAN

* Méthodes de prédiction
    * Random Forest
    * SVM linéaire


## <font color='blue'>Analyse exploratoire des données</font>
* A traiter: statistiques des features retenues
## <font color='blue'>Score RFM</font>
* Score RFM : nouvelle feature dans le dataset
* Suppression des features issues du process RFM
* Suppression du pays
## <font color='blue'>PCA</font>
* Analyse syntaxique de la description  des produits.
* Représetation 2D ou 3D des clusters
* Application du t-SNE pour réductiond de dimension
* Application du DBSAN pour le clustering


# <font color='blus'> Synthèse du 05/07/2018</font>

* Parties du dataset

* Classes d'un client : table d'une transaction a une table client.==> groupby

* Variables vont caractérisier les clients.

* Description : faire des expressions régulieres pour détcter des motifs pertinents. Predre des mots clés spécifiques et construire des variables à partir de ces mots clés.

* Fréquence de mots clés : atteindre un seuil.

* Identifiant de deux mots similaires.

* Outliers : éliminer.

* Redondance : voire les corrélations.

* Essayer d'autres méthodes pour le clustering. Comparer avec des données.


## <font color='blue'>Analyse exploratoire des données</font>
* A traiter: statistiques des features retenues
## <font color='blue'>Score RFM</font>
* Score RFM : nouvelle feature dans le dataset
* Suppression des features issues du process RFM
* Suppression du pays
## <font color='blue'>PCA</font>
* Analyse syntaxique de la description  des produits.
* Représetation 2D ou 3D des clusters
* Application du t-SNE pour réductiond de dimension
* Application du DBSAN pour le clustering


# <font color='blus'> Synthèse du 21/06/2018</font>
## <font color='blue'>Questions</font>
* Classification + Prédiction?
* Séries temporelles?
* Séparation jeux de données : entraînement / test : modifier les critères de séparation des 2 sets?
  * horizon de temps pour le train. Entrainement sur les 10 premiers mois et prédire sur les deux mois suivants.
* **Variables qui identifient le comportement d'achat des clients :**
    * Pays
    * Identifiant de la marchandise achetée
    * Volume de la marchandise
    * Chiffre d'affaires
    * Période d'achat dans l'année
* Dataleakage total=unit__price*unit : ce n'est pas du data-leakage car multplication de deux valeurs
* Invoice date:  permet de retirer des informations sur le compotement du client.
* Classer des le premeir achaat : la frequence ne permet pas de faire une classification pour un nouveau client. Une fois l'achat fait, on classfie le client pour pouvoir fire de la pub en ligne.
* Les clients dans cette bas efont très peu d'achat. Verifier par un graphe.
* Un client pourrait changer de classe.
* Pays autres que UK : certains pays n'ont pas Amazone.
* Descriiption: donne des informations sur les articles. Frequence des mots, conjonctions a éliminer. Voire la lib <font color='blue'>**nltk**</font> qui gère du texte. Enlever les mots de la description pour iosseler le produit. Permet de classer les types de produits.

In [ ]:
import pandas as pd
import numpy as np
import time

import p3_util
import p3_util_plot
import p5_util
import p5_util_plot

# <font color='blues'>Acquisition des données</font>

On s'est assuré qu'i n'y avait qu'un seul onglet dans le fichier.

In [ ]:
fileName = './data/OnlineRetail.xlsx'
t0 = time.time()
df_invoice = pd.read_excel(fileName)
t1 = time.time()
print("Elapsed time = %1.2f" %(t1-t0))

In [ ]:
df_invoice_save = df_invoice.copy()

## <font color='blue'>Data cleaning</font>

 * Les lignes contenant les customers ID a Nan sont purgées.
 * Les lignes contenant des variables aberrantes sont purgées

In [ ]:
df_invoice = df_invoice_save.copy()
print(df_invoice.shape)
df_invoice.sample(3)

**Description des variables quantitatives**

In [ ]:
df_invoice.describe()

* Drop rows containing Nan values

In [ ]:
rows_before = df_invoice.shape[0]
print(df_invoice.shape)
df_invoice.dropna(axis=0, how='any', inplace=True)
print(df_invoice.shape)
rows_after = df_invoice.shape[0]
print("NUll values : Percent of dropped rows = %1.2F" %(np.abs(rows_before-rows_after)*100/rows_before))

* Drop quantity <= 0

In [ ]:
rows_before = df_invoice.shape[0]
df_invoice = df_invoice[(df_invoice['Quantity']>0)]
rows_after = df_invoice.shape[0]
print("CustomerID <= 0 : Percent of dropped rows = %1.2F" %(np.abs(rows_before-rows_after)*100/rows_before))

* Drop CustomerID <= 0

In [ ]:
df_invoice = df_invoice[df_invoice['CustomerID'] > 0]
print(df_invoice.shape)

df_invoice = df_invoice[df_invoice['UnitPrice']>0]
print(df_invoice.shape)

df_invoice = df_invoice[df_invoice['Quantity']>0]
print(df_invoice.shape)

* Verification de l'existance de valeurs nulles dans chacune des colonnes et chacune des lignes

In [ ]:
# Verification sur toutes les lignes (axis=1) puis somme des résultats en colonne (axis=0)
df_invoice.isnull().sum(axis=1).sum(axis=0)

In [ ]:
df_invoice_save = df_invoice.copy()

## <font color='blue'>Data preparation</font>

In [ ]:
df_invoice = df_invoice_save.copy()
df_invoice.sample(1)

### <font color='blue'>Distribution des pays dans le dataset</font>

In [ ]:
import p3_util_plot
dict_to_display = dict()
for country in df_invoice.Country.unique() :
    dict_to_display[country] = df_invoice[df_invoice['Country']==country].shape[0]


title="Distribution de l\'origine des transactions"
xlabel = "Pays"
ylabel = "Nb de transactions"
p3_util_plot.df_display_dict_hist(dict_to_display, title, xlabel, ylabel, fontSize=12, color='blue')

** Filtrage des pays les moins représentés**

In [ ]:
list_countries_keep = ['EIRE','France','Germany','United Kingdom']
list_countries_keep = ['United Kingdom']

df_invoice_new = pd.DataFrame()
for country in list_countries_keep : 
    df_invoice_new = df_invoice_new.append(df_invoice[df_invoice['Country']==country], ignore_index=True)

print(df_invoice_new.shape)
rows_after = df_invoice_new.shape[0]
print("Countries filtering : Percent of dropped rows = %1.2F" %(np.abs(rows_before-rows_after)*100/rows_before))

In [ ]:
df_invoice = df_invoice_new.copy()
df_invoice_new = None

### <font color='blue'>For any invoice, Total price is added in feature *Total*</font>

In [ ]:
if 'Total' in df_invoice.columns :
    del(df_invoice['Total'])

df_invoice['Total'] = df_invoice.Quantity * df_invoice.UnitPrice
df_invoice = df_invoice[df_invoice['Total']>0]
df_invoice.sample(2)

### <font color='blue'>Oultliers in feature *Total*</font>

In [ ]:
import p5_util_plot

p5_util_plot.p5_df_subplot(df_invoice[['Quantity','UnitPrice','Total']],\
                           type_plot='box', is_outliers_removed = True, tuple_reshape=(1,3))

** Purge des outliers total **

In [ ]:
import p3_util
zmin, zmax = p3_util.df_boxplot_min_max(df_invoice , 'Total')

print(df_invoice.shape)
df_invoice = df_invoice[df_invoice['Total']>=zmin]

print(df_invoice.shape)

df_invoice = df_invoice[df_invoice['Total']<=zmax]
print(df_invoice.shape)


## <font color='blue'>Représentation des variables qualitatives</font>

# <font color='blue'>RFM scoring for each customer</font>

** Invoice date are converted into seconds**

In [ ]:
df_invoice['InvoiceDate'] = df_invoice['InvoiceDate'].apply(p5_util.p5_convert_timestamp_to_days)

In [ ]:
df_invoice_save = df_invoice.copy()
df_invoice.sample(1)

In [ ]:
df_invoice = df_invoice_save.copy()
df_invoice.sample(1)

In [ ]:
df_invoice.CustomerID.isnull().any()

** Values from feature CustomerID are casted into integer**

In [ ]:
df_invoice.CustomerID = df_invoice.CustomerID.astype(int)

** For each customer, RFM score is computed from followings features: <font color=blue>InvoiceNo, InvoiceDate, Total</font> **

In [ ]:
day_now = df_invoice.InvoiceDate.max()
print("Day now = {}".format(day_now))

In [ ]:
df_RFM = pd.DataFrame(df_invoice.groupby('CustomerID').agg({'InvoiceNo': lambda x: len(x)}))
df_RFM['Recency'] = df_invoice.groupby('CustomerID').agg({'InvoiceDate': lambda x: (day_now - x.max())})
df_RFM['Monatary'] = df_invoice.groupby('CustomerID').agg({'Total': lambda x: x.sum()})

In [ ]:
df_RFM.rename(columns={'InvoiceNo':'Frequency'}, inplace=True)

** Scoring Frequency, Monetary and Recency from quantile values limmits **

In [ ]:
# Limit of quantile values are computed from 
df_rfm_quantiles = df_RFM.quantile(q=[0.25,0.5,0.75])

df_rfm_quantiles.reset_index(inplace=True)

df_rfm_quantiles.rename(index={0:'Q1',1:'Q2',2:'Q3'},inplace=True)

del(df_rfm_quantiles['index'])

In [ ]:
df_rfm_quantiles

In [ ]:
df_rfm_quantiles.loc['Q1','Frequency']

** For applying same scoring function, <font color='blue'>*Recency*</font> values are applied with opposite sign.**

In [ ]:
df_RFM['neg_recency'] = df_RFM['Recency'].apply(lambda x: -x)
df_rfm_quantiles['neg_recency'] = df_rfm_quantiles['Recency'].apply(lambda x: -x)

In [ ]:
# Classification des clients en score RFM
df_RFM['R_score'] = df_RFM.neg_recency.apply(p5_util.func_rfm_scoring, args=(df_rfm_quantiles,'neg_recency'))
df_RFM['F_score'] = df_RFM.Frequency.apply(p5_util.func_rfm_scoring, args=(df_rfm_quantiles,'Frequency'))
df_RFM['M_score'] = df_RFM.Monatary.apply(p5_util.func_rfm_scoring, args=(df_rfm_quantiles,'Monatary'))

In [ ]:
df_RFM.head(5)

** Aggregation du scoring FRM **

In [ ]:
df_RFM['RFM'] = df_RFM['R_score'].map(str)+df_RFM['F_score'].map(str)+df_RFM['M_score'].map(str)

In [ ]:
df_RFM.reset_index(inplace=True)

In [ ]:
df_RFM_restricted = df_RFM[['CustomerID','RFM']]

In [ ]:
df_RFM_restricted.sample()
df_invoice.sample()

In [ ]:
if 'RFM' in df_invoice.columns :
    del(df_invoice['RFM'])
df_RFM_restricted.sample(1)

In [ ]:
df_merged = pd.merge(df_invoice, df_RFM_restricted, how='left', on=['CustomerID'])

** Features used for RFM computation are droped; this avoid data-leak**

In [ ]:
df_merged = df_merged[['Description','Quantity','UnitPrice','CustomerID','RFM']]

df_merged.sample(5)

** First analysis : we do not take into account Description; CustomerID is processed as row index **

In [ ]:
df_merged.index= df_merged.CustomerID
df_invoice_1 = df_merged[['Quantity','UnitPrice','RFM']]

df_invoice_1.sample(1)

In [ ]:
df_merged.sample(1)

## <font color='blue'>Customers segmentation issue from clustering</font>

** One hot encoding du score RFM **

In [ ]:
df_invoice_1.shape

In [ ]:
from sklearn import preprocessing

encoder = preprocessing.OneHotEncoder() 
try :
    encoded_RFM = encoder.fit_transform(df_invoice_1.RFM.values.reshape(-1,1))
except ValueError as valueError :
    print("\n*** Erreur encodage : {}".format(valueError))
encoded_RFM.shape

** Standardisation des données **

In [ ]:
df_invoice_1.columns

In [ ]:
from sklearn import preprocessing
# ------------------------------------------------
# Quantitative values are extracted from dataframe
# ------------------------------------------------
X_quantitative_std = df_invoice_1[['Quantity','UnitPrice']].values

# ------------------------------------------------
# Quantitative values are casted as float
# ------------------------------------------------
X_quantitative_std = X_quantitative_std.astype(float)

# ------------------------------------------------
# Data scaling
# ------------------------------------------------
std_scale = preprocessing.StandardScaler().fit(X_quantitative_std)
X_quantitative_std = std_scale.transform(X_quantitative_std)

** Aggrégation des données **

In [ ]:
import scipy
X_std = scipy.sparse.hstack((X_quantitative_std, encoded_RFM))

In [ ]:
import seaborn as sns

df_corr = pd.DataFrame(X_std.toarray()).corr()
df_corr.shape
_z = sns.heatmap(df_corr, annot=False)

Les features obtenues par le score RFM sont décorrélées les unes des autres.

### <font color='blue'>Dimension shift </font>

#### <font color='blue'>PCA Analysis</font>

In [ ]:
import p3_util_plot

df_std = pd.DataFrame(X_std.toarray())
z__ = p3_util_plot.df_pca_all_plot(df_std, plot=True)

La variance du modèle est expliquée par l'ensemble des features. 

Une réduction de dimension sur la base de l'ACP n'est pas pertinente.


#### <font color='blue'>Dimension shift with t-SNE</font>

**tSNE allows to build separate groups of data points based on probability points of same group 
belongs to same neighborhood.**

**This is also an easy way to apply clustering based on this dimension reduction.**

** Sampling of 3000 random data points and 30 data points**

** Verification de l'intersection vide entre la liste des samples et de la liste des 3000 data points **

In [ ]:
set_intersec = set(list_random_index_sample).intersection(list_random_index)
print("\n Intersection : nombre d'éléments dans les des deux listes = {0}".format(len(set_intersec)))

** Sampling arrays are extracted from index list **

In [ ]:
print(X_std.toarray().shape)
nb_rows = X_std.toarray().shape[0]
X_std_sample = X_std.toarray()[random_index]
X_std_sample_2 = X_std.toarray()[random_index_sample]
print(X_std_sample.shape,X_std_sample_2.shape)

** t-SNE is applied for dimension shifting **

In [ ]:
import p5_util
my_list_tsne_perplexity=[var for var in range(5,55,5)]
dict_tsne_result = p5_util.tsne_2D_process_perplexity(X_std_sample, tsne_iter=3000, list_tsne_perplexity=my_list_tsne_perplexity)

In [ ]:
import p5_util_plot
nb_col = 4     
p5_util_plot.plot_2D_dict_tsne_result(dict_tsne_result, nb_col)

#### <font color='blue'>Data-shift for samples of 30 data points </font>

## <font color='blue'>Clustering over a sample</font>

### <font color='blue'>Among all tSNE results, compute Kmeans clustering and get cluster scores</font>

** Display all Kmeans scores **

In [ ]:
import p5_util

n_cluster_start = 50
n_cluster_end = 100
n_cluster_optimum,best_perplexity,score_max = p5_util.scan_best_kmean_cluster_from_tsne_result(dict_tsne_result\
                                                                                               , n_cluster_start\
                                                                                               , n_cluster_end)

In [ ]:
print("\n *** Kmeans : Optimal cluster count= {0} / Silhouette = {1:0.3F} / tSNE Perplexity = {2}\n".format(n_cluster_optimum, score_max, best_perplexity))
n_cluster_kmean = n_cluster_optimum


** Compute and display optimal clustering **

In [ ]:
nclusters = n_cluster_optimum
X_std_projected = dict_tsne_result[best_perplexity]

cluster_kmean = KMeans(n_clusters=nclusters).fit(X_std_projected)
centers_kmean = cluster_kmean.cluster_centers_
preds_kmean_sample = cluster_kmean.predict(X_std_projected)
my_title = str(nclusters)+" Kmeans clusters"
p3_util_plot.clustering_plot(X_std_projected, preds_kmean_sample, nclusters, title=my_title\
                             , X_center=None)

In [ ]:
my_title = str(nclusters)+" Kmeans clusters with extra sampling"
p3_util_plot.clustering_plot(X_std_projected, preds_kmean_sample, nclusters, title=my_title\
                             , X_center=X_std_projected_2)

In [ ]:
import p5_util

n_cluster_start = 50
n_cluster_end = 100
n_cluster_gmm_optimum,best_perplexity_gmm,score_max_gmm = p5_util.scan_best_gmm_cluster_from_tsne_result(dict_tsne_result\
                                                                                            , n_cluster_start\
                                                                                            , n_cluster_end)


In [ ]:
print("\n *** GMM : Optimal cluster count= {0} / Silhouette = {1:0.3F} / tSNE Perplexity = {2}\n".format(n_cluster_gmm_optimum,score_max_gmm,best_perplexity_gmm))

### <font color='blue'>DBSCAN over sample</font>

In [ ]:
import p3_util_plot

labels_trained_dbscan, labels_test_dbscan = p3_util_plot.clustering_dbscan_plot_and_metrics(X_std_sample_train\
                                                                                            ,X_std_sample_test\
                                                                                            ,parameter_eps=0.3\
                                                                                            ,parameter_min_samples=20)

In [ ]:
from sklearn.cluster import DBSCAN
dict_dbscan_scoring = dict()

for dbscan_eps in np.logspace(-5, 1, 100).astype(np.float32) :
    dbscan = DBSCAN(eps=dbscan_eps, min_samples=5).fit(X_std_sample)
    clustering_dbscan = dbscan.fit_predict(X_std_sample)
    #dict_dbscan_scoring[] = clustering_dbscan.labels_
    #dbscan_nclusters = len(set(clustering_dbscan.labels_)) - (1 if -1 in clustering_dbscan.labels_ else 0)
    dbscan_nclusters = len(set(clustering_dbscan)) - (1 if -1 in clustering_dbscan else 0)

    print("DBSCAN : estimation du nombre de clusters : {0} dbscan_eps = {1:1.6F}".format(dbscan_nclusters, dbscan_eps))

    if dbscan_nclusters <= 1 :
        print("\nERROR : DBSCAN cluster bruité!\n")
    else :
        dict_dbscan_scoring[dbscan_nclusters] = silhouette_score(X_std_sample, clustering_dbscan)



In [ ]:
score_max = 0
cluster_optimum = 0
for n_cluster in dict_dbscan_scoring.keys() :
    score = dict_dbscan_scoring[n_cluster]
    print ("DBSCAN : cluster count = {0} / Average silhouette score is : {1:0.3F}".format(n_cluster, score))
    if score > score_max : 
        score_max = score
        n_cluster_optimum = n_cluster

print("\n *** Optimal cluster count= {0} / Silhouette = {1:0.3F}\n".format(n_cluster_optimum, score_max))        

### <font color='blue'>Target variable depending from best clustering process</font>

In [ ]:
X = dict_tsne_result[best_perplexity]

In [ ]:
from sklearn.cluster import KMeans
cluster_kmean = KMeans(n_clusters=n_cluster_kmean).fit(X)

# Predict the cluster for each data point
preds_kmean = cluster_kmean.predict(X)

#Find the cluster centers
centers_kmean = cluster_kmean.cluster_centers_
print(preds_kmean.shape)


# <font color='blus'>Classement des items Description</font>

In [ ]:
df_merged_save = df_merged.copy()
df_invoice_1_save = df_invoice_1.copy()
X_std_save = X_std.copy()

In [ ]:
df_invoice = df_merged.copy()

In [ ]:
df_merged.columns

In [ ]:
df_merged.shape

## <font color='blue'>Using NLTK to create additional features from *Description*</font>

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk import text

import string

In [ ]:
#Punctuation exclude characters '-' and '_'

# Each-one of the values from Description feature is cleaned from punctuation 
#--------------------------------------------------------------------
#
#--------------------------------------------------------------------
def get_my_punctuation():
    my_punctuation = str()
    list_punctuation_excluded = ['-','_',' ']
    for char in string.punctuation:
        if char in list_punctuation_excluded:
            pass
        else:
            my_punctuation += char
    return my_punctuation
#--------------------------------------------------------------------
    
#--------------------------------------------------------------------
#
#--------------------------------------------------------------------
def cb_remove_punctuation(item, list_char_remove) :
    item_no_punctuation = [ char for char in item.lower() if char not in list_char_remove ]
    item_no_punctuation = "".join(item_no_punctuation)
    return item_no_punctuation
#--------------------------------------------------------------------

#--------------------------------------------------------------------
#
#--------------------------------------------------------------------
def cb_remove_stopwords(item) :
    list_word = item.split()
    item_no_stopwords = [ word for word in list_word if word.lower() not in stopwords.words('english') ]
    item_no_stopwords = " ".join(item_no_stopwords)
    return item_no_stopwords.upper()
#--------------------------------------------------------------------

#--------------------------------------------------------------------
#
#--------------------------------------------------------------------
def cb_stemmer(item, stemmer):
    list_word = item.split()
    stemmed_item = [ stemmer.stem(word.lower()) for word in list_word ]
    stemmed_item = " ".join(stemmed_item)
    return stemmed_item.upper()
#--------------------------------------------------------------------

#--------------------------------------------------------------------
#
#--------------------------------------------------------------------
def cb_lemmatizer(item, lemmatizer):
    list_item = item.split()
    lemmatized_item = [ lemmatizer.lemmatize(word.lower()) for word in list_item ]
    lemmatized_item = " ".join(lemmatized_item)
    return lemmatized_item.upper()
#--------------------------------------------------------------------
    
#--------------------------------------------------------------------
#
#--------------------------------------------------------------------
def cb_clean_numeric_word_in_item(item) :
    list_word = item.split()
    list_cleaned = [ word for word in list_word if not (word.isnumeric())]
    new_item = ' '.join(list_cleaned)
    return new_item
#--------------------------------------------------------------------

#--------------------------------------------------------------------
#
#--------------------------------------------------------------------
def cb_clean_list_word(item, list_no_word):
    list_word = item.lower().split()
    #list_no_word is converted as lower characters
    no_word_lower = ' '.join(list_no_word).lower()
    list_no_word = no_word_lower.split()
    list_cleaned = [ word for word in list_word if word.lower() not in list_no_word ]
    new_item = ' '.join(list_cleaned)
    return new_item.upper()
#--------------------------------------------------------------------    
    
    
#--------------------------------------------------------------------
#
#--------------------------------------------------------------------
def get_ser_set_len(ser):
    return len(set([ item for item in ser ]))
#--------------------------------------------------------------------


**Let's consider all items into *Description* feature as a corpus.**

**Remove any punctuation from any *Description* feature**

In [ ]:

df_invoice.Description.iloc[0] +=",!.&~"
item = df_invoice.Description.iloc[0]
print(item)

my_punctuation = get_my_punctuation()
df_invoice.Description = df_invoice.Description.apply(cb_remove_punctuation,args=(my_punctuation,) )

item = df_invoice.Description.iloc[0]
print(item)
print(get_ser_set_len(df_invoice.Description))

**We remove stopwords in orde to extract words with most information.**

In [ ]:
df_invoice.Description = df_invoice.Description.apply(cb_remove_stopwords)
print(get_ser_set_len(df_invoice.Description))

In [ ]:
df_invoice_save = df_invoice.copy()

In [ ]:
df_invoice = df_invoice_save.copy()
df_invoice.sample(2)

#### Lemmatization of *Description* values

In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
df_invoice.Description = df_invoice.Description.apply(cb_lemmatizer,args=(lemmatizer,))
print(get_ser_set_len(df_invoice.Description))

#### Stemming of *Description* values

In [ ]:
stemmer = SnowballStemmer('english')
df_invoice.Description = df_invoice.Description.apply(cb_stemmer,args=(stemmer,))
print(get_ser_set_len(df_invoice.Description))

#### Clean all numeric word from item Description 

In [ ]:
df_invoice.Description = df_invoice.Description.apply(cb_clean_numeric_word_in_item)
print(get_ser_set_len(df_invoice.Description))

#### Clean repetitives keyword from item Description 

In [ ]:
print(get_ser_set_len(df_invoice.Description))
list_no_words=['SET','PACK']
df_invoice.Description = df_invoice.Description.apply(cb_clean_list_word, args=(list_no_words,))
print(get_ser_set_len(df_invoice.Description))

##### *df_invoice.Description* is regarded as a corpus.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer( min_df=1, max_df=.8, ngram_range=(1,1))
matrix_weights = vectorizer.fit_transform(df_invoice.Description)
print(matrix_weights.shape)

### Clustering

In [ ]:
import scipy

hstack_matrix = scipy.sparse.hstack((X_std, matrix_weights))
hstack_matrix.shape,matrix_weights.shape, encoded_RFM.shape

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

n_cluster=6
cluster_kmean = KMeans(n_clusters=n_cluster).fit(hstack_matrix)

# Predict the cluster for each data point
preds_kmean = cluster_kmean.predict(hstack_matrix)

#### <font color='blue'>Building *nltk.Text* object</font>

In [ ]:
if True :
    list_items = list()
    for item in df_invoice.Description:
        list_items.append(item)

if False :
    my_Text = text.Text(new_item)
else :
    my_Text = text.Text(list_items)
    print(len(list_items))
    set_items = set(list_items)
    print(len(set_items))


In [ ]:
my_Text = text.Text(df_invoice.Description)
fdist = nltk.FreqDist(my_Text)

In [ ]:
fdist.most_common()[-10:-1],fdist.most_common()[:10]

In [ ]:
# Contextes d'utilisation d'un mot

In [ ]:
my_Text.concordance("SET")

In [ ]:
my_Text.similar("SET")

In [ ]:
my_Text.collocations()

In [ ]:
my_Text.similar("KEEP")

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,15))
fdist.plot(10000, cumulative=True)

#### <font color='blue'>Valeurs de la feature *Description* qui contiennent le mot *SET*</font>

In [ ]:
#----------------------------------------------------
#
#----------------------------------------------------
def is_word_in_item(item, word) :
    if word in item :
        return item
    else :
        return None
#----------------------------------------------------
#----------------------------------------------------
#
#----------------------------------------------------
def clean_numeric_word_in_item(item) :
    list_word = item.split(' ')
    for word in list_word:
        if word.isnumeric():
            list_word.remove(word)
        else :
            pass
    
    # Rebuild word from list
    new_item = ""
    for word in list_word :
        new_item+=word
        new_item+=" "
    # Remove trailer character
    new_item = new_item[:-1] 
    
    return new_item
#----------------------------------------------------
result_SET = None
result_SET = df_invoice.Description.apply(is_word_in_item, args=('MEDIUM',))
result_SET = df_invoice.Description.apply(clean_numeric_word_in_item)

result_SET.dropna(how='any', inplace=True)

result_SET.head(10)

### <font color='blue'>Using random Forests for RFM score prediction classification</font>

**Classification used into previous step is used for targeting model.***

In [ ]:
y_sample = preds_kmean.copy()
hstack_matrix.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(hstack_matrix, y_sample, test_size=0.1)
rfc = RandomForestClassifier(n_estimators=500)
rfc_model = rfc.fit(X_train, y_train)
y_pred = rfc_model.predict(X_test)

In [ ]:
print("Précision de la prédiction par random forest: {0:1.3F}".format(accuracy_score(y_test, y_pred)))

## <font color='blue'>Using linear SVC for prediction</font>

In [ ]:
from sklearn  import svm
svm_model = svm.LinearSVC()

svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

In [ ]:
print("Précision de la prédiction par Linear SVC: {0:1.3F}".format(svm_model.score(X_test,y_test)))

# <font color='blue'>Using computed model for prediction </font>